In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [5]:
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
navegador.maximize_window()

In [14]:
def busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo):
    #entrar no google
    navegador.get('https://www.google.com/')
    
    #tratar valores inseridos
    produto = produto.lower()
    lista_termos_produtos = produto.split(" ")
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    
    # adicionar o produto na busca do google
    navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    #procura pelo shopping google
    navegador.find_element(By.XPATH,'//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()
        
    #pega lista de resultados do google shopping
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    lista_ofertas =[]
    
    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME,'XrAfOe').text
        preco = preco.split('\n')
        preco = preco[1]
        preco = preco.replace(' Custava R$ 789,00','').replace(' Custava R$ 1.149,00','').replace(' Custava R$ 10.923,85','').replace(' Custava R$ 1.279,00','').replace(' Custava R$ 1.199,00','')
        preco = preco.replace(' Custava R$ 125,58','').replace(' Custava R$ 509,00','').replace(' Custava R$ 1.009,00','')
        preco = preco.replace(' Custava R$ 1.799,00','')
        preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
        preco = float(preco)
        
        nome = resultado.find_element(By.CLASS_NAME,'tAxDx').text
        nome = nome.lower()
        
        #verificando o nome 
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos =True
                
        tem_todos_termos_produto = True
        for palavra in lista_termos_produtos:
            if palavra not in nome:
                tem_todos_termos_produto = False
                
        # se tem_temosPbanidos = False e tem_todos_termos_produtos = True -- executa a analise
        preco_maximo = float(preco_maximo)
        preco_minimo = float(preco_minimo)
        if preco_minimo <= preco <= preco_maximo:
            elemento_link = resultado.find_element(By.CLASS_NAME,'aULzUe')
            elemento_pai = elemento_link.find_element(By.XPATH,'..')
            link = elemento_pai.get_attribute('href')
            lista_ofertas.append((nome,preco,link))
    return lista_ofertas

In [15]:
produto = 'Smartphone Samsung Galaxy'
termos_banidos = 'A53'
preco_minimo = 800
preco_maximo = 6000

lista_produtos = busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo)

In [16]:
print(lista_produtos)

[('smartphone samsung galaxy a13 128gb 4gb ram - preto', 1034.1, 'https://www.google.com/url?url=https://www.magazineluiza.com.br/smartphone-samsung-galaxy-a13-128gb-4g-tela-6-6-dual-4gb-ram-camera-quadrupla-selfie-8mp-preto/p/cc2kace52h/te/ga13/%3F%26seller_id%3Dsemprevoce&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwil7oHv8Zz8AhXVqpUCHdQ1A5oQguUECJsQ&usg=AOvVaw10Ny-8iRyxswSljHQwdz_R'), ('smartphone samsung galaxy a53 5g, 128gb, 8gb ram, câmera quádrupla, tela infinita ...', 1979.1, 'https://www.google.com/url?url=https://www.casasbahia.com.br/smartphone-samsung-galaxy-a53-5g-128gb-8gb-ram-camera-quadrupla-tela-infinita-de-65-quot-azul-1536303711/p/1536303711%3Futm_medium%3DCpc%26utm_source%3Dgoogle_freelisting%26IdSku%3D1536303711%26idLojista%3D209633%26tipoLojista%3D3P&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwil7oHv8Zz8AhXVqpUCHdQ1A5oQguUECKsQ&usg=AOvVaw2I7GyN10IRZZbMlIbOHY9b'), ('smartphone galaxy a03 64gb 4gb ram preto - samsung', 1044.05, 'https://www.google.com/url?url=http://www.nagem.com.br/produto

In [23]:
def busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    navegador.get("https://www.buscape.com.br/")
    
    # pesquisar pelo produto no buscape
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(5)
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_Text__h_AF6.Text_MobileHeadingS__Zxam2').text
            nome = resultado.find_element(By.CLASS_NAME,'Text_Text__h_AF6.Text_MobileLabelXs__ER_cD.Text_DesktopLabelSAtLarge__baj_g.SearchCard_ProductCard_Name__ZaO5o').text
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas

In [30]:
produto = 'iphone 11 64 gb'
termos_banidos = 'mini watch'
preco_minimo = 2000
preco_maximo = 12000

lista_ofertas_busca = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

In [31]:
print(lista_ofertas_busca)

[('smartphone apple iphone 11 64gb câmera dupla', 2930.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 vermelho 64gb câmera dupla', 2998.45, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-vermelho-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 pro max 64gb câmera tripla', 4297.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-pro-max-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 usado 64gb câmera dupla', 2559.92, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-usado-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 pro max usado 64gb câmera tripla', 3800.72, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-pro-max-usado-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 pro usado 64gb câmera tripla', 